In [17]:
import pandas as pd
import glob as glob
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import optimize
%matplotlib inline
sns.set_style('white')

### Set parameters

In [23]:
N_samples = 1000
threshold = .5 
REWARD = 1

### Free parameters to fit

In [3]:
COST = 0.5
B=6
A=60 
u_skill = .625 #mean of beta
tao_skill = 20 #variance of beta
skill=.4
scale = 200

### Functions

In [19]:
def prob_reward(e,diff,skill, A, B):
    return 1/(1 + A * np.exp(-(e * skill / diff) * B)) 

def get_effort(reward, cost, diff, skill, A, B):
    Us=list()
    es = np.linspace(0, 1, 100) 
    for i in es:
        U=reward*prob_reward(i, diff, skill, A, B)-cost*i #utility function
        Us.append(U)
    return es[np.argmax(Us)], Us, es  #return argmax of utility, and utilities 

def your_eff_know_skill(reward, cost, diff,skill, A, B):  #model for planning. Takes in diff, skill, and cost -returns effort
    sampled_effort= list()
    sampled_skill=list()
    for i in xrange(N_samples):
        d = np.random.choice(diff)
        effort,_,_=get_effort(reward, cost, d, skill, A, B)
        sampled_effort.append(effort)
        sampled_skill.append(skill)
    
    return sampled_effort

def create_priors(N_samples, tao_skill,u_skill, A, B, COST):
    output = {
        'sampled_skill_high':[],'sampled_diff_high':[],'sampled_skill_low':[],'sampled_diff_low':[],
        'sampled_skill_low_fail':[],' sampled_diff_low_fail':[],'sampled_diff_low_fail':[],'sampled_skill_high_fail':[],
        'sampled_diff_high_fail':[],'diff_samp':[],'skill_samp':[],'effort_samp':[],'success_samp':[]}
    for _ in xrange(N_samples):
        skill = np.random.beta(tao_skill*u_skill,(1-u_skill)*tao_skill) #skill is a beta function
        diff = np.random.beta(.5,.5) #difficulty is a beta function
        effort,_,_ = get_effort(REWARD, COST, diff, skill, A, B) 
        high_effort = effort > threshold
        success = prob_reward(effort,diff,skill, A, B) > np.random.random()

        output['diff_samp'].append(diff)
        output['skill_samp'].append(skill)
        output['effort_samp'].append(effort)
        output['success_samp'].append(success)
        if success and high_effort: 
            output['sampled_skill_high'].append(skill)
            output['sampled_diff_high'].append(diff)
        elif success and not high_effort: 
            output['sampled_skill_low'].append(skill)
            output['sampled_diff_low'].append(diff)
        elif not success and not high_effort: 
            output['sampled_skill_low_fail'].append(skill)
            output['sampled_diff_low_fail'].append(diff)
        elif not success and high_effort: 
            output['sampled_skill_high_fail'].append(skill)
            output['sampled_diff_high_fail'].append(diff)

    return output

def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

### Function that takes in free parameters and spits out model predictions

In [32]:
def run_model(cost, A, B, u_skill,tao_skill, skill, scale):
    REWARD=1
    efforthigh_success=list()
    effortlow_success=list()
    efforthigh_fail = list()
    effortlow_fail=list()
    effortbaseline =list()
    priors= create_priors(1000, tao_skill,u_skill, A,B,cost)
    effort_high_success = your_eff_know_skill(REWARD, cost, priors['sampled_diff_high'],skill, A,B) 
    efforthigh_success.append(effort_high_success)
    effort_low_success = your_eff_know_skill(REWARD, cost, priors['sampled_diff_low'],skill,A,B) 
    effortlow_success.append(effort_low_success)
    effort_high_fail = your_eff_know_skill(REWARD, cost, priors['sampled_diff_high_fail'],skill,A,B) 
    efforthigh_fail.append(effort_high_fail)
    effort_low_fail = your_eff_know_skill(REWARD, cost, priors['sampled_diff_low_fail'],skill,A,B) 
    effortlow_fail.append(effort_low_fail)
    effort_baseline = your_eff_know_skill(REWARD, cost, np.random.beta(.5, .5, size=100),skill,A,B)
    effortbaseline.append(effort_baseline)
    
    d = {'Model' : pd.Series([np.mean(efforthigh_success)*scale, np.mean(effortlow_success)*scale, np.mean(efforthigh_fail)*scale, np.mean(effortlow_fail)*scale, np.mean(effortbaseline)*scale], index=['High Effort Success', 'Low Effort Success', 'High Effort Failure', 'Low Effort Failure',"Baseline"]),
     'Human' : pd.Series([107, 57, 30, 35,86], index=['High Effort Success', 'Low Effort Success', 'High Effort Failure', 'Low Effort Failure',"Baseline"])}
    df = pd.DataFrame(d)
    rmse_val = rmse( df['Model'],df['Human'])
    return rmse_val

In [33]:
def wrapped_run_model(x):
    return run_model(*x)

In [34]:
inputs = np.array([0.5,60,6, .625, 20, .4, 200])
%timeit wrapped_run_model(inputs)
output

1 loop, best of 3: 2.35 s per loop


22.149375623851721

In [12]:
cost = 0.5
B=6
A=60 
u_skill = .625 #mean of beta
tao_skill = 20 #variance of beta
skill=.4
scale = 200
threshold = .5
priors=create_priors(1000, tao_skill, u_skill,  A, B, cost)


### Optminize function

In [38]:
key_words=dict(cost = 0.5,B=6,A=60 ,u_skill = .625 ,tao_skill = 20, skill=.4, scale =200)
rmse_val= run_model(**key_words)


In [50]:
inputs = np.array([0.5,60,6, .625, 20, .4, 200])
minimum = optimize.fmin(wrapped_run_model, inputs, maxiter =10)

In [52]:
minimum

array([   0.50270981,   59.3608913 ,    6.2575177 ,    0.62219193,
         20.10839234,    0.40216785,  200.10932945])

In [13]:


d = {'Model' : pd.Series([np.mean(efforthigh_success)*scale, np.mean(effortlow_success)*scale, np.mean(efforthigh_fail)*scale, np.mean(effortlow_fail)*scale, np.mean(effortbaseline)*scale], index=['High Effort Success', 'Low Effort Success', 'High Effort Failure', 'Low Effort Failure',"Baseline"]),
     'Human' : pd.Series([107, 57, 30, 35,85], index=['High Effort Success', 'Low Effort Success', 'High Effort Failure', 'Low Effort Failure',"Baseline"])}

df = pd.DataFrame(d)
df

,Human,Model
High Effort Success,107,100.380202
Low Effort Success,57,58.324444
High Effort Failure,30,32.468687
Low Effort Failure,35,2.952121
Baseline,85,56.404242
